# 数据预处理

就我个人的理解和经验来说，无论Machine Learning还是Deep Learning，模型预测的准确性实际上更加依赖于数据是否处理得**合理、干净**。

因此，本章节主要来学习机器学习的基础部分，**数据预处理**。

参考网页：

https://scikit-learn.org/stable/modules/preprocessing.html

https://scikit-learn.org.cn/view/123.html

数据预处理主要依赖于sklearn.preprocessing包，对于不同的模型、不同经济意义或者物理意义的数据，都有不同的数据处理方式。

“通常，学习算法受益于数据集的标准化。如果数据集中存在一些异常值，则更适合使用健壮的缩放器或转换器。不同缩放器，转换器和规范化器在包含边缘异常值的数据集上的行为突出显示了**比较不同缩放器对含有异常值数据的效果**。”

## 1 Standardization, or mean removal and variance scaling

## 标准化或均值去除和方差缩放

标准化过程的本质是，把那些分布不均匀的数据集，按照各种计算方式，将其转换为**均值为0，标准差为1**的分布。

在实践中，我们通常会忽略分布的形状，而只是通过删除每个特征的平均值来实现特征数据中心化，然后除以非常数特征的标准差来缩放数据。

### 1.1 sklearn.preprocessing.scale

sklearn.preprocessing.scale(X, *, axis=0, with_mean=True, with_std=True, copy=True)

| 参数     | 说明 |
|----------|------|
| **X** | {array-like, sparse matrix} 数据要居中和缩放。 |
| **axis** | int (0 by default) 用于计算平均值和标准偏差的轴。如果为0，则独立标准化每个特征，否则（如果为1）则标准化每个样本。 |
| **with_mean** | boolean, True by default 如果为True，则在缩放之前将数据居中。 |
| **with_std** | boolean, True by default 如果为True，则将数据缩放到单位方差（或等效地，单位标准偏差）。 |
| **copy** | boolean, optional, default True 设置为False将执行就地标准化并避免复制（如果输入已经是numpy数组或scipy.sparse CSC矩阵，并且轴为1）。 |


In [4]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
X_scaled = preprocessing.scale(X_train)

X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [5]:
X_scaled.mean(axis=0) # 缩放后的样本均值

array([0., 0., 0.])

In [6]:
X_scaled.std(axis=0) # 缩放后的样本标准差

array([1., 1., 1.])

### 1.2 sklearn.preprocessing.StandardScaler

class sklearn.preprocessing.StandardScaler(*, copy=True, with_mean=True, with_std=True)

| 参数           | 说明                                                                                         |
| -------------- | -------------------------------------------------------------------------------------------- |
| `copy`         | **boolean, optional, default True**<br>如果为False，请尝试避免复制并改为就地缩放。不能保证总是在原地工作；例如，如果数据不是NumPy数组或scipy.sparse CSR矩阵，则可能仍会返回副本。 |
| `with_mean`    | **boolean, True by default**<br>如果为True，则在缩放之前将数据居中。尝试使用稀疏矩阵时，这不起作用（并且会引发异常），因为将它们居中需要建立一个密集的矩阵，在通常的使用情况下，该矩阵可能太大而无法容纳在内存中。 |
| `with_std`     | **boolean, True by default**<br>如果为True，则将数据缩放到单位方差（或等效地，单位标准偏差）。 |

---

| 属性             | 说明                                                                                         |
| ---------------- | -------------------------------------------------------------------------------------------- |
| `scale_`         | **ndarray or None, shape (n_features,)**<br>每个要素的数据相对缩放。这是使用np.sqrt（var）计算的。当with_std = False时等于无。<br>0.17版本中的新功能：scale |
| `mean_`          | **ndarray or None, shape (n_features,)**<br>训练集中每个特征的平均值。当with_mean = False时等于无。 |
| `var_`           | **ndarray or None, shape (n_features,)**<br>训练集中每个要素的方差。用于计算scale_。当with_std = False时等于无。 |
| `n_samples_seen_`| **int or array, shape (n_features,)**<br>估计器为每个要素处理的样本数。如果不缺少样本，则n_samples_seen将为整数，否则将为数组。将在新的调用中重置为fit，但在partial_fit调用中递增。 |


In [4]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]

data

# print(scaler.mean_)

# print(scaler.transform(data))

# print(scaler.transform([[2, 2]]))

[[0, 0], [0, 0], [1, 1], [1, 1]]

In [5]:
scaler = StandardScaler()
scaler.fit(data) # 用data来计算均值、方差等指标

StandardScaler()

In [6]:
scaler.mean_

array([0.5, 0.5])

In [10]:
scaler.var_

array([0.25, 0.25])

In [7]:
scaler.transform(data)

array([[-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.]])

In [8]:
#可以直接使用训练集对测试集数据进行转换
scaler.transform([[2, 2]])

array([[3., 3.]])

<font color='red' size=3> **scale和StandardScale区别：**

`scale()`函数和`StandardScaler()`函数都可以把数据标准化，处理的过程都是 `(X-mean) / std`。

但是又有点不同，`scale()` 不能迁移到新的数据集，如果是处理训练集和测试集，只能是把训练集和测试集合起来，计算出共同的`mean`和`std`，然后 `(X-mean) / std`，再分成训练集和测试集。**这里的 mean 和 std 的计算涉及到了测试集，是训练集和测试集共同的期望和方差**。

而 `StandardScaler()` 可以迁移到新的数据集，只需要处理训练集，拿训练集的数据计算出均值 `x_train_mean`、`x_train_std` 和方差，然后训练集的 `X_train` 和测试集的 `X_test` 都执行标准化。**这里注意：测试集的标准化是利用的训练集的均值和方差**。也就是说 **假设训练集的期望和测试集的期望是一样的，这样只需要计算出训练集的期望之后，直接用于测试集就可以了，这里只是训练集的期望和方差，没有涉及测试集**。

> 在机器学习中，我们是从整体中采用抽样的方式抽出训练集，这意味着我们默认，这部分训练集可以代替整体，也就是说训练集的期望就是整体的期望。测试集标准化的时候，它的期望采用的正是训练集的期望。因此 `StandardScaler()` 才是我们经常用的方式。
